In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

1、读取数据集

In [3]:
listings = pd.read_csv('../data/listings.csv.gz')

2、数据集预处理

In [4]:
listings['price'] = listings.price.str.replace('$', '').str.replace(',', '')
listings['price'] = listings.price.astype(float)
listings = listings.loc[(listings.price <= 600) & (listings.price > 0)]

In [5]:
listings.amenities = listings.amenities.str.replace('[{}]', '').str.replace('"', '')

In [6]:
columns = [
    'host_is_superhost', 
    'host_identity_verified', 
    'host_has_profile_pic',
    'is_location_exact',
    'requires_license',
    'instant_bookable',
    'require_guest_profile_picture',
    'require_guest_phone_verification'
]
for col in columns:
    listings[col] = listings[col].replace('f', 0, regex=True)
    listings[col] = listings[col].replace('t', 1, regex=True)

In [7]:
listings['security_deposit'] = listings['security_deposit'].fillna(value=0)
listings['security_deposit'] = listings['security_deposit'].replace('[\$,)]', '', regex=True)
listings['security_deposit'] = listings['security_deposit'].astype(float)

listings['cleaning_fee'] = listings['cleaning_fee'].fillna(0)
listings['cleaning_fee'] = listings['cleaning_fee'].replace('[\$,)]', '', regex=True)
listings['cleaning_fee'] = listings['cleaning_fee'].astype(float)


In [ ]:
new_columns = columns + ['security_deposit', 'cleaning_fee', 'host_listings_count', 'host_total_listings_count', 'minimum_nights', 'bathrooms', 'bedrooms', 'guests_included', 'number_of_reviews', 'review_scores_rating', 'price']

In [10]:
list(listings.columns)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',
